In [1]:
from rt_utils import RTStructBuilder
import matplotlib.pyplot as plt
import numpy as np
import pydicom
import os

In [2]:
root_path  = "//zkh/appdata/RTDicom/Projectline - Modelling lung cancer outcomes [panama code]/DICOM_data/DATA_VOLLEDIG_unstructured/"
PxList = np.sort(os.listdir(root_path))

In [3]:
def FindCT_UID(path):
    files = os.listdir(path)
    ct_dicom_dataset = pydicom.dcmread(os.path.join(path,files[0]))
    series_instance_uid = ct_dicom_dataset[(0x0020, 0x0052)].value
    return series_instance_uid

In [4]:
def FindRT_UID_perContour(path,roi_name_to_find):
    files = os.listdir(path)
    rtstruct_dataset = pydicom.dcmread(path+files[0])
    if rtstruct_dataset.Modality == "RTSTRUCT":
        roi_sequence = rtstruct_dataset.StructureSetROISequence
        for roi_item in roi_sequence:
            roi_number = roi_item.ROINumber
            roi_name = roi_item.ROIName
            if roi_name == roi_name_to_find:
                # Print information for the found ROI
                #print(f"ROI Name: {roi_name}")
                #print(f"ROI Number: {roi_number}")
                referenced_frame_of_reference_uid = roi_item.ReferencedFrameOfReferenceUID
                #print(f"Referenced Frame of Reference UID: {referenced_frame_of_reference_uid}")
                return referenced_frame_of_reference_uid

        else:
            return None


In [5]:
def FindUIDMatch(RtPaths,CTPaths,labels):
    numcontours=0
    for currRTpath in RTStruct_fpaths:
        for currCTpath in CTPaths:
            for label in labels:
                ctUID = FindCT_UID(currCTpath)
                rtUID = FindRT_UID_perContour(currRTpath,label)
                if rtUID == ctUID and ctUID is not None and rtUID is not None:
                    numcontours+=1                    
                    #print("Match",currCTpath)
    return numcontours

In [6]:
def GetTheContourRTUTILS(ctfolder,rtfolder,label):
    if len(ctfolder)==0:
        print("No CT")
    for k in range(len(ctfolder)):
        for j in range(len(rtfolder)):
            rtfiles = os.listdir(rtfolder[j])
            for i in range(len(rtfiles)):
                try:
                    rtstruct = RTStructBuilder.create_from(
                        dicom_series_path=ctfolder[k],
                        rt_struct_path=str(rtfolder[j]+rtfiles[i])
                    )
                    print(rtstruct.get_roi_names())
                except Exception as e:
                    print(e)
                    #continue

        #mask_3d = rtstruct.get_roi_mask_by_name(label)

In [ ]:
for Px in PxList:
    AvgCt_fpaths = []
    RTStruct_fpaths = []
    BP50_CT_fpaths,BP60_CT_fpaths= ([],[])
    BP10_CT_fpaths,BP90_CT_fpaths= ([],[])
    num50GTVs,numITVs,num60GTVs = (0,0,0)
    #Look for the Paths
    for root, dirs, files in os.walk(root_path+Px, topdown=True):
        for d in dirs:
            if "Thorax" in d or "Ave" in d or "MIP" or "ct" in d.lower():
                AvgCt_fpaths.append(root+'/'+d+'/')
            if ('RTSTRUCT' in d or "pproved" in d) or ("PinnPlan" in d and not("RTDOSE" in d)):
                RTStruct_fpaths.append(root+'/'+d+'/')
    #GET the UIDs
    ITVlabels = ['ITV',"IGTV","IgTV"]
    GetTheContourRTUTILS(BP50_CT_fpaths,RTStruct_fpaths,GTVlabels)
    GetTheContourRTUTILS(BP60_CT_fpaths,RTStruct_fpaths,GTVlabels)
    #numITVs = FindUIDMatch(RTStruct_fpaths,AvgCt_fpaths,ITVlabels)
    #num50GTVs = FindUIDMatch(RTStruct_fpaths,BP50_CT_fpaths,GTVlabels)
    #num60GTVs = FindUIDMatch(RTStruct_fpaths,BP60_CT_fpaths,GTVlabels)
    print("Px:",Px,"ITVs Matched:",numITVs,"GTV 50%:",num50GTVs,"GTV 60%:",num60GTVs)
    if False and num50GTVs != 0 and num60GTVs != 0:
        break

Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.2.840.113619.2.25.4.1433992366.5660.1532595603.1977
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.2.840.113619.2.25.4.1433992366.5660.1532595603.1977
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.2.840.113619.2.25.4.1433992366.5660.1532595603.1977
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.2.840.113619.2.25.4.1433992366.5660.1532595603.1977
Px: 0004879 ITVs Matched: 0 GTV 50%: 0 GTV 60%: 0
No CT
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.2.840.113619.2.278.3.663816714.255.1430916283.324.1
Px: 0010539 ITVs Matched: 0 GTV 50%: 0 GTV 60%: 0
No CT
Loaded RTStruct re

Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.2.840.113619.2.25.4.1433992366.6151.1530183122.2123
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.2.840.113619.2.25.4.1433992366.6151.1530183122.2123
Px: 0222671 ITVs Matched: 0 GTV 50%: 0 GTV 60%: 0
No CT
No CT
Px: 0225780 ITVs Matched: 0 GTV 50%: 0 GTV 60%: 0
No CT
No CT
Px: 0228708 ITVs Matched: 0 GTV 50%: 0 GTV 60%: 0
No CT
No CT
Px: 0234628 ITVs Matched: 0 GTV 50%: 0 GTV 60%: 0
No CT
No CT
Px: 0247019 ITVs Matched: 0 GTV 50%: 0 GTV 60%: 0
No CT
No CT
Px: 0248219 ITVs Matched: 0 GTV 50%: 0 GTV 60%: 0
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000017070606173793600011863
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance 

No CT
No CT
Px: 0546680 ITVs Matched: 0 GTV 50%: 0 GTV 60%: 0
No CT
No CT
Px: 0549920 ITVs Matched: 0 GTV 50%: 0 GTV 60%: 0
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000017052407242146900012646
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000017052407242146900012646
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000017052407242146900012646
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000017052407242146900012646
Px: 0558328 ITVs Matched: 0 GTV 50%: 0 GTV 60%: 0
No CT
No CT
Px: 0561128 ITVs Matched: 0 GTV 50%: 0 GTV 60%: 0
Loaded RTStruct references image(s) that are not contained i

Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000019010408083132600001597
Px: 0664779 ITVs Matched: 0 GTV 50%: 0 GTV 60%: 0
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000016061506224339100001974
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000016061506224339100001974
Px: 0681369 ITVs Matched: 0 GTV 50%: 0 GTV 60%: 0
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000018101106164240200004792
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000018101106164240200004792
['GTVtumor', 'ITV']


Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000016102008474761900005997
Px: 0868262 ITVs Matched: 0 GTV 50%: 0 GTV 60%: 0
No CT
No CT
Px: 0868569 ITVs Matched: 0 GTV 50%: 0 GTV 60%: 0
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000016102407433786400004162
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000016102407433786400004162
Px: 0869737 ITVs Matched: 0 GTV 50%: 0 GTV 60%: 0
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000016122908062371600005181
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.

...

Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.2.840.113619.2.25.4.1433992366.1968.1519914573.2324
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.2.840.113619.2.25.4.1433992366.1968.1519914573.2324
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.2.840.113619.2.25.4.1433992366.1008.1517478194.1653
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.2.840.113619.2.25.4.1433992366.1008.1517478194.1653
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.2.840.113619.2.25.4.1433992366.1968.1519914573.2324
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.2.840.11

Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.2.840.113619.2.25.4.1433992366.9528.1531389983.1419
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.2.840.113619.2.25.4.1433992366.9528.1531389983.1419
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.2.840.113619.2.25.4.1433992366.9528.1531389983.1419
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.2.840.113619.2.25.4.1433992366.9528.1531389983.1419
Px: 5341373 ITVs Matched: 0 GTV 50%: 0 GTV 60%: 0
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.2.840.113619.2.25.4.1433992366.1993.1521192471.2225
Loaded RTStruct references image(s) that are not contained in input series data.

Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.2.840.113619.2.25.4.1433992366.515.1534413087.2206
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.2.840.113619.2.25.4.1433992366.515.1534413087.2206
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.2.840.113619.2.25.4.1433992366.515.1534413087.2206
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.2.840.113619.2.25.4.1433992366.515.1534413087.2206
Px: 5670144 ITVs Matched: 0 GTV 50%: 0 GTV 60%: 0
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000018091706341401600003195
Loaded RTStruct references image(s) that are not contained in input series data. P

Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000018061205585088900009423
Px: 5826774 ITVs Matched: 0 GTV 50%: 0 GTV 60%: 0
No CT
No CT
Px: 5835644 ITVs Matched: 0 GTV 50%: 0 GTV 60%: 0
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.2.840.113619.2.25.4.1433992366.3292.1517392973.1888
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.2.840.113619.2.25.4.1433992366.4503.1518424455.1853
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.2.840.113619.2.25.4.1433992366.3292.1517392973.1888
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.2.840.113619.2.25.4.1433992366.4503.1518424455.1853
Loaded RTStruct 

Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000018061306140156700012703
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000018061306140156700012506
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000018061306140156700012506
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000018061306140156700012506
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000018061306140156700012703
Px: 6425154 ITVs Matched: 0 GTV 50%: 0 GTV 60%: 0
Loaded RTStruct references image(s) that are not contained in input se

In [59]:
BP60_CT_fpaths[0]

'//zkh/appdata/RTDicom/Projectline - Modelling lung cancer outcomes [panama code]/DICOM_data/DATA_VOLLEDIG_unstructured/0023840\\6134912.1/4D thorax 2.0 2.0 I30f 3 0 - 100  TRIGGER_DELAY 60%/'